In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [56]:
# 각 년도별 엑셀 파일을 하나의 데이터 프레임으로 만들기
df=pd.read_excel('시군구별 월별/시군구별 전력사용량(2004년).xlsx')
df=df.rename(columns=df.iloc[1]).drop(index=[0,1]).reset_index(drop=True)
for i in range(5,23):
    data=pd.read_excel('시군구별 월별/시군구별 전력사용량(20'+'%02d'%(i)+'년).xlsx')
    data=data.rename(columns=data.iloc[1]).drop(index=[0,1]).reset_index(drop=True)
    df=pd.concat([df,data],axis=0)
# df

In [57]:
# 합계만 추출해 각 시군구 월별 전력 사용량 데이터 추출
df=df[df['계약종별']=='합 계'].drop(columns='계약종별').reset_index(drop=True)

In [58]:
# 잘못된 데이터 제거

# 1. 잘못된 지역 제거
df=df[df['시도']!='황해북도']

# 2. 음수인 데이터와 0인 데이터 보간 (양 끝 값은 다음 달이나 이전 달 값으로)
df=df.applymap(lambda x:np.nan if str(x)<="0" else x)
df.iloc[:,3:]=df.iloc[:,3:].interpolate(axis=1).fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)
# df

In [59]:
# 필요한 칼럼만 추출 
df_percent=df.iloc[:,:3]

# 월별 데이터 연도별로 변환하기 위해 더하기
df_percent['전력 사용량']=df.iloc[:,4:].sum(axis=1)

# 비율 구하기 위해 각 연도의 전체 전력 사용량 계산
df_sum=df_percent.groupby('연도').sum()
df_sum.rename(columns={'전력 사용량':'총 전력 사용량'},inplace=True)

# 전체 사용량 df에 붙이기
df_percent=df_percent.merge(df_sum,on='연도')

# 비율 구하기
df_percent['비율']=df_percent['전력 사용량']/df_percent['총 전력 사용량']

In [60]:
df_percent

,연도,시도,시군구,전력 사용량,총 전력 사용량,비율
0,2014,서울특별시,종로구,1.548716e+09,4.360669e+11,0.003552
1,2014,서울특별시,중구,2.294425e+09,4.360669e+11,0.005262
2,2014,서울특별시,용산구,1.219836e+09,4.360669e+11,0.002797
3,2014,서울특별시,성동구,1.690394e+09,4.360669e+11,0.003876
4,2014,서울특별시,동대문구,1.280193e+09,4.360669e+11,0.002936
...,...,...,...,...,...,...
2062,2022,경상남도,거창군,4.381278e+08,4.981305e+11,0.000880
2063,2022,경상남도,합천군,3.875468e+08,4.981305e+11,0.000778
2064,2022,제주특별자치도,제주시,3.282972e+09,4.981305e+11,0.006591
2065,2022,제주특별자치도,서귀포시,2.209029e+09,4.981305e+11,0.004435
